In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('mol2vec.csv')
df.replace(to_replace='#NAME?',value=np.nan,inplace=True)
df.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = df.columns
df = df.astype("float", errors = "ignore")
df = np.float32(df)
df[np.isnan(df)] = 0
df[np.isinf(df)] = 0
df=pd.DataFrame(df,columns=columns)
ncol = df.shape[1]-1
x = df.iloc[:,0:ncol]
y = df['active']
corr_matrix = x.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
x1 = x.drop(to_drop, axis=1)
x2 = x1.loc[:, (x1 != x1.iloc[0]).any()] 
x2 = x2.round(6)
import warnings
warnings.filterwarnings("ignore")

In [2]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
0,-3.042170,-3.151222,-3.232676,7.560959,5.113686,-0.568981,-19.256578,-4.969994,16.522644,1.793195,...,12.655949,20.450172,4.086627,-10.276544,-2.190987,-1.115333,-9.388687,-15.132810,-4.057350,1.0
1,-4.727887,-4.351881,-4.658237,-1.792979,-0.499149,-8.724982,-8.886705,5.068110,-1.706909,-1.742328,...,14.233682,-0.425539,-0.464745,-10.994161,-12.039774,-12.723747,-4.175037,-11.076891,2.639213,1.0
2,-4.097823,-3.983348,-5.248144,-0.722321,-1.265023,-8.232691,-7.873246,5.745502,-0.900785,-1.008448,...,13.890897,-0.387179,0.128263,-11.509965,-10.279533,-11.952808,-3.907961,-10.316779,2.601157,1.0
3,-5.616269,-6.542370,-6.168895,-3.166161,1.509026,-11.890589,-10.365218,4.797748,-1.633804,-2.156420,...,16.483412,1.412684,-0.536551,-12.108947,-14.011013,-14.057915,-5.047316,-14.451323,1.707880,1.0
4,-4.396163,-5.507332,-5.355181,-2.798300,1.738860,-10.982598,-10.979731,4.270193,-1.071588,-2.923557,...,16.781738,-0.069494,-0.499126,-10.858052,-13.415128,-14.032432,-5.018421,-12.879906,1.733685,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,-0.245050,0.000723,-1.600506,2.734911,1.502080,0.887623,-5.741633,0.586250,4.324068,-0.374940,...,4.085058,5.202066,2.300924,-5.023169,-0.517164,-1.643741,-2.033808,-4.831588,0.884858,0.0
2020,-1.848185,-1.777678,-1.303566,0.651581,1.617647,1.635964,-2.599049,-0.967981,0.539179,1.095203,...,1.162264,3.851595,-0.020075,-1.346415,-1.949291,-0.575638,0.865116,-2.355083,-1.106393,0.0
2021,0.009402,0.458835,2.047133,-0.661602,1.160643,-0.072311,-4.937521,-2.287562,4.538611,-0.010992,...,1.278592,3.704251,0.990562,-2.375479,0.759892,-2.489954,-2.155487,-2.489589,-1.650008,0.0
2022,-3.098453,-1.974883,-2.323134,-0.688243,8.347803,0.660212,-16.231108,-3.275765,4.571203,-5.276364,...,3.041915,10.691944,3.452311,-7.664165,-4.948164,-1.304520,-8.855995,-8.327632,-5.991015,0.0


In [3]:
from sklearn.model_selection import train_test_split
x_train,x_val_test,y_train,y_val_test = train_test_split(x2,y,test_size = 0.2,random_state = 2)
x_val, x_test, y_val, y_test = train_test_split(x_val_test,y_val_test,test_size = 0.5,random_state = 2)

In [4]:
train = pd.concat([x_train,y_train],axis=1)
train

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
515,4.250311,-1.049513,-4.572200,8.028474,2.177597,1.179794,-13.136718,-0.994468,6.370598,2.910774,...,8.233146,13.554775,-0.343263,-8.689811,-3.972875,-8.765325,-6.503958,-12.608653,-2.436182,1.0
1794,0.652956,-0.614835,-2.400780,3.325422,1.038107,-0.206821,-3.949853,-0.080500,2.941349,0.763702,...,3.964174,4.012534,-0.353810,-2.910851,-1.525682,-2.240588,-1.033048,-4.831987,-0.668356,0.0
333,-0.701988,-0.925419,-2.398040,4.294267,3.381105,-0.228989,-10.575158,0.072078,8.718456,-1.697693,...,6.319836,10.438110,0.574847,-4.702792,-3.140366,-1.681350,-4.518072,-10.627499,-4.077444,1.0
734,6.900640,-7.507542,-5.940346,11.763398,-1.152488,1.303991,-15.393419,-0.056893,9.547030,4.587806,...,15.467580,14.663936,-4.125107,-12.531883,-8.119540,-10.433078,-5.034236,-18.337982,-1.850323,1.0
1753,1.580982,1.759796,-2.368860,1.097806,-2.500142,2.089177,-7.927256,2.025424,0.972917,3.273584,...,6.664584,6.173492,-2.812610,-5.184052,-0.791000,-3.125652,-3.586450,-4.168434,-0.479909,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,0.956570,0.435979,-2.886325,0.913237,-1.118360,0.614024,-6.919654,0.146735,2.351073,2.582623,...,5.448304,5.666750,-0.686185,-4.072188,-1.590533,-1.811071,-3.047822,-5.328300,-0.935039,0.0
1608,0.520243,-0.148544,-2.803409,3.693946,-0.824370,0.180679,-7.239314,0.685743,7.059150,3.268087,...,5.333204,6.882022,1.416558,-1.902741,0.567567,-2.995536,0.802698,-3.136844,-0.113226,0.0
493,3.160671,-5.298172,-6.038879,13.380636,-1.891510,2.656222,-12.937065,-1.016984,12.397815,4.920306,...,12.571790,16.436958,-0.881875,-7.836222,-2.206282,-3.649037,-4.292634,-14.818405,-0.932096,1.0
527,3.047690,-4.513142,-2.752672,10.715731,1.133903,1.314588,-14.639387,-1.325667,9.472692,1.506177,...,9.842552,15.004856,-0.781079,-6.614820,-0.349379,-5.979506,-3.669782,-12.582667,-1.518844,1.0


In [5]:
val = pd.concat([x_val,y_val],axis=1)
val

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
1230,-0.211411,-1.221566,-1.984175,1.181452,0.461747,0.069131,-2.992733,-0.539408,1.572320,0.362728,...,2.665218,3.042087,-0.313533,-3.149515,-1.561416,-1.988872,-0.186135,-2.856934,-1.505045,0.0
1255,1.022214,-3.356968,-4.912842,-1.777337,1.136152,-2.593746,-9.545589,-3.208907,7.167033,1.531969,...,13.007440,6.640826,1.837897,-5.754990,-5.542613,-7.925042,-7.699146,-12.944720,1.285944,0.0
268,4.061099,-2.080274,-9.022748,5.410938,4.013893,1.740186,-14.704828,-0.401675,5.528722,2.714361,...,13.480717,13.336947,-0.382278,-11.359468,-6.554892,-7.428294,-4.620588,-17.770733,-1.355362,1.0
1922,1.097949,0.026803,-1.661032,2.518736,-1.115905,1.100074,-5.545524,0.522725,2.128078,1.579599,...,4.780176,4.767006,-2.324328,-3.723465,-0.834663,-1.797622,-1.907156,-3.956598,-1.358670,0.0
1568,0.461504,-1.109219,-0.565499,9.537881,0.725491,-2.785860,-6.879022,-0.999697,8.192760,-0.006587,...,2.527103,8.300374,-1.063042,-2.357727,-2.271389,-6.277322,1.294907,-4.052553,-1.549841,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,5.652952,-5.752883,-5.637353,6.958992,-1.544493,0.056667,-17.225447,-0.954423,9.343005,9.609364,...,15.237697,12.306355,-1.192498,-13.731361,-5.242792,-8.972743,-5.845655,-16.455872,-2.773660,1.0
1206,0.308810,-1.806311,-1.380667,5.359597,-1.432222,0.510763,-4.479892,0.461367,5.060540,3.008818,...,5.320620,6.327936,-0.024665,-2.490691,-0.574782,-1.700048,-0.311626,-3.937521,-0.181744,0.0
352,0.859086,-5.665197,-3.503780,8.132488,-0.338263,-2.213161,-13.433861,-0.321296,8.715608,1.802377,...,14.529463,11.252768,-0.669202,-7.887460,-4.531750,-5.199586,-4.617962,-11.682182,-1.302563,1.0
1780,-0.559792,-1.705862,-3.156123,1.670572,-0.680508,-3.141714,-2.350225,1.803738,1.418282,0.379769,...,4.109841,2.094606,-0.430338,-4.564408,-1.171204,-1.548956,-0.791011,-4.697448,0.147174,0.0


In [24]:
test1 = pd.concat([x_test,y_test],axis=1)
test1

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
1710,-0.588799,0.826047,0.264683,-2.574159,3.544894,-0.459080,-1.482318,-0.007915,1.442779,-0.321173,...,1.176665,3.506747,0.609200,-1.772735,-0.598777,-1.322951,-1.197879,-2.191543,-3.172286,0.0
532,2.667033,-10.827370,-2.896134,1.390352,-3.353129,-9.597544,-13.425303,1.173387,7.233574,0.328978,...,17.514051,8.104202,-1.130868,-10.039116,-11.598745,-8.777007,-6.845168,-15.686932,3.559965,1.0
494,0.227661,-1.350179,-4.743934,7.770904,1.586193,1.390250,-13.710514,-1.927295,5.307272,5.392054,...,9.517765,10.916759,-2.962256,-8.843686,-5.479238,-8.045248,-1.951520,-11.290153,-1.748865,1.0
1008,2.763614,-1.078457,-2.542609,3.325340,-1.154039,1.663332,-5.029128,0.525910,2.932755,2.562090,...,3.713113,6.757771,-0.857754,-3.067784,-0.474282,-4.059687,-0.990283,-5.136946,-0.157337,1.0
1810,1.220878,-1.380341,-1.373073,5.168466,-1.011778,0.639889,-4.217169,0.129338,2.988985,0.267203,...,4.104524,4.936094,-0.142244,-1.629248,-1.502120,-1.711085,-0.567609,-4.437906,0.030621,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,-4.425474,-6.206540,-5.609136,-2.300187,1.630253,-12.226448,-12.571782,3.887144,-1.435319,-3.725411,...,17.110325,0.942102,-0.750876,-10.007064,-15.421255,-13.159926,-5.055804,-14.510889,1.976946,1.0
3,-5.616268,-6.542370,-6.168895,-3.166161,1.509026,-11.890589,-10.365218,4.797748,-1.633804,-2.156420,...,16.483412,1.412684,-0.536551,-12.108947,-14.011013,-14.057915,-5.047316,-14.451323,1.707880,1.0
920,0.181228,0.562082,-0.432312,1.180502,-0.390850,0.103073,-1.353564,0.892218,0.718385,0.876947,...,0.406859,0.020669,0.377933,0.009263,0.258876,-0.904654,-1.010721,-0.986637,0.172122,1.0
874,6.260910,-5.140882,-1.785725,14.647200,-1.665310,0.494829,-11.735408,-1.220797,5.492476,0.433948,...,10.476008,13.224262,-2.362246,-4.118767,-7.593261,-5.234460,-5.183010,-13.351004,-2.456605,0.0


In [7]:
col_name = train.columns.values.tolist()
col_name.pop()

'active'

In [8]:
from pycaret.classification import *
CM = setup(data = train, target = 'active', test_data = val, silent=True, numeric_features = col_name,
             session_id=0, fold_strategy = 'stratifiedkfold', fold = 5, fold_shuffle = True, normalize = True, normalize_method = 'minmax')

,Description,Value
0,session_id,0
1,Target,active
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(1619, 300)"
5,Missing Values,False
6,Numeric Features,299
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [9]:
mlp = create_model('mlp')
tunedmlp = tune_model(mlp,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedmlp)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8549,0.9162,0.8571,0.8289,0.8428,0.7082,0.7086
1,0.8086,0.8904,0.7397,0.8182,0.7770,0.6102,0.6125
2,0.7994,0.8804,0.8151,0.7580,0.7855,0.5975,0.5989
3,0.8519,0.9143,0.9247,0.7849,0.8491,0.7055,0.7147
4,0.8266,0.9072,0.8219,0.8000,0.8108,0.6509,0.6511
Mean,0.8283,0.9017,0.8317,0.7980,0.8130,0.6545,0.6571
Std,0.0223,0.0140,0.0602,0.0251,0.0291,0.0463,0.0477


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,MLP Classifier,0.8366,0.9044,0.9277,0.7404,0.8235,0.6750,0.6899


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.596673,0.871356,0.763590,0.339436,0.229405,0.713469,0.915850,0.766661,0.246346,0.610858,...,0.179938,0.865118,0.902924,0.897048,0.893266,0.947896,0.421826,0.0,0.0,0.9178
1,0.639353,0.816179,0.667370,0.238672,0.248595,0.651654,0.833907,0.694445,0.301305,0.651882,...,0.240837,0.815005,0.826968,0.772842,0.794202,0.828693,0.519998,0.0,1.0,0.7803
2,0.744488,0.849168,0.532340,0.483475,0.330479,0.752260,0.769390,0.770387,0.285212,0.693368,...,0.177992,0.707211,0.807656,0.783236,0.834795,0.771666,0.427091,1.0,1.0,0.8173
3,0.641973,0.903613,0.774207,0.384978,0.184514,0.737401,0.883928,0.795394,0.251806,0.653553,...,0.123020,0.854079,0.916789,0.901050,0.870574,0.934901,0.426975,0.0,0.0,0.8851
4,0.619954,0.874259,0.810200,0.624021,0.236910,0.647194,0.867252,0.754209,0.311381,0.597900,...,0.158722,0.880347,0.889378,0.807319,0.912795,0.933768,0.420251,0.0,0.0,0.6354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.799560,0.754270,0.643566,0.536195,0.172319,0.713179,0.737869,0.755434,0.322681,0.935287,...,0.155058,0.661590,0.832689,0.750921,0.818642,0.787203,0.377203,1.0,1.0,0.8114
198,0.614671,0.856247,0.783418,0.481726,0.175513,0.723720,0.897253,0.793734,0.280612,0.703699,...,0.188115,0.877789,0.921747,0.903092,0.891612,0.935127,0.468373,0.0,0.0,0.8512
199,0.633709,0.756536,0.713664,0.576160,0.206641,0.660488,0.785284,0.772561,0.316518,0.661370,...,0.169870,0.773990,0.846254,0.829869,0.834830,0.843611,0.428948,1.0,1.0,0.8333
200,0.584620,0.858842,0.725086,0.356093,0.196903,0.638933,0.923885,0.830049,0.244833,0.611456,...,0.176632,0.837904,0.910368,0.906253,0.885291,0.926147,0.479942,0.0,0.0,0.9074


In [10]:
xgb = create_model('xgboost')
tunedxgb = tune_model(xgb,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedxgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8426,0.9405,0.8980,0.7857,0.8381,0.6863,0.6921
1,0.8117,0.9096,0.8699,0.7515,0.8063,0.6251,0.6314
2,0.8241,0.9109,0.8425,0.7834,0.8119,0.6471,0.6486
3,0.8519,0.9398,0.9110,0.7917,0.8471,0.7048,0.7113
4,0.8390,0.9151,0.8767,0.7901,0.8312,0.6781,0.6815
Mean,0.8339,0.9232,0.8796,0.7805,0.8269,0.6683,0.6730
Std,0.0142,0.0140,0.0237,0.0148,0.0155,0.0285,0.0291


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8465,0.9302,0.9036,0.7653,0.8287,0.6914,0.6993


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.596673,0.871356,0.763590,0.339436,0.229405,0.713469,0.915850,0.766661,0.246346,0.610858,...,0.179938,0.865118,0.902924,0.897048,0.893266,0.947896,0.421826,0.0,0.0,0.9657
1,0.639353,0.816179,0.667370,0.238672,0.248595,0.651654,0.833907,0.694445,0.301305,0.651882,...,0.240837,0.815005,0.826968,0.772842,0.794202,0.828693,0.519998,0.0,1.0,0.8659
2,0.744488,0.849168,0.532340,0.483475,0.330479,0.752260,0.769390,0.770387,0.285212,0.693368,...,0.177992,0.707211,0.807656,0.783236,0.834795,0.771666,0.427091,1.0,1.0,0.9848
3,0.641973,0.903613,0.774207,0.384978,0.184514,0.737401,0.883928,0.795394,0.251806,0.653553,...,0.123020,0.854079,0.916789,0.901050,0.870574,0.934901,0.426975,0.0,0.0,0.9989
4,0.619954,0.874259,0.810200,0.624021,0.236910,0.647194,0.867252,0.754209,0.311381,0.597900,...,0.158722,0.880347,0.889378,0.807319,0.912795,0.933768,0.420251,0.0,1.0,0.5502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.799560,0.754270,0.643566,0.536195,0.172319,0.713179,0.737869,0.755434,0.322681,0.935287,...,0.155058,0.661590,0.832689,0.750921,0.818642,0.787203,0.377203,1.0,1.0,0.9958
198,0.614671,0.856247,0.783418,0.481726,0.175513,0.723720,0.897253,0.793734,0.280612,0.703699,...,0.188115,0.877789,0.921747,0.903092,0.891612,0.935127,0.468373,0.0,0.0,0.9940
199,0.633709,0.756536,0.713664,0.576160,0.206641,0.660488,0.785284,0.772561,0.316518,0.661370,...,0.169870,0.773990,0.846254,0.829869,0.834830,0.843611,0.428948,1.0,1.0,0.9924
200,0.584620,0.858842,0.725086,0.356093,0.196903,0.638933,0.923885,0.830049,0.244833,0.611456,...,0.176632,0.837904,0.910368,0.906253,0.885291,0.926147,0.479942,0.0,0.0,0.8753


In [11]:
rf = create_model('rf')
tunedrf = tune_model(rf, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedrf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8488,0.9434,0.8367,0.8311,0.8339,0.6951,0.6951
1,0.8210,0.9103,0.8356,0.7821,0.8079,0.6407,0.6419
2,0.8272,0.8989,0.8151,0.8041,0.8095,0.6513,0.6514
3,0.8642,0.9314,0.8630,0.8400,0.8514,0.7264,0.7266
4,0.8328,0.9173,0.8014,0.8239,0.8125,0.6617,0.6619
Mean,0.8388,0.9203,0.8304,0.8162,0.8230,0.6750,0.6754
Std,0.0157,0.0156,0.0210,0.0208,0.0170,0.0315,0.0313


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8465,0.9323,0.8554,0.7889,0.8208,0.6870,0.6887


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.596673,0.871356,0.763590,0.339436,0.229405,0.713469,0.915850,0.766661,0.246346,0.610858,...,0.179938,0.865118,0.902924,0.897048,0.893266,0.947896,0.421826,0.0,0.0,0.9223
1,0.639353,0.816179,0.667370,0.238672,0.248595,0.651654,0.833907,0.694445,0.301305,0.651882,...,0.240837,0.815005,0.826968,0.772842,0.794202,0.828693,0.519998,0.0,0.0,0.5013
2,0.744488,0.849168,0.532340,0.483475,0.330479,0.752260,0.769390,0.770387,0.285212,0.693368,...,0.177992,0.707211,0.807656,0.783236,0.834795,0.771666,0.427091,1.0,1.0,0.7951
3,0.641973,0.903613,0.774207,0.384978,0.184514,0.737401,0.883928,0.795394,0.251806,0.653553,...,0.123020,0.854079,0.916789,0.901050,0.870574,0.934901,0.426975,0.0,0.0,0.9910
4,0.619954,0.874259,0.810200,0.624021,0.236910,0.647194,0.867252,0.754209,0.311381,0.597900,...,0.158722,0.880347,0.889378,0.807319,0.912795,0.933768,0.420251,0.0,0.0,0.5560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.799560,0.754270,0.643566,0.536195,0.172319,0.713179,0.737869,0.755434,0.322681,0.935287,...,0.155058,0.661590,0.832689,0.750921,0.818642,0.787203,0.377203,1.0,1.0,0.9588
198,0.614671,0.856247,0.783418,0.481726,0.175513,0.723720,0.897253,0.793734,0.280612,0.703699,...,0.188115,0.877789,0.921747,0.903092,0.891612,0.935127,0.468373,0.0,0.0,0.9634
199,0.633709,0.756536,0.713664,0.576160,0.206641,0.660488,0.785284,0.772561,0.316518,0.661370,...,0.169870,0.773990,0.846254,0.829869,0.834830,0.843611,0.428948,1.0,1.0,0.8419
200,0.584620,0.858842,0.725086,0.356093,0.196903,0.638933,0.923885,0.830049,0.244833,0.611456,...,0.176632,0.837904,0.910368,0.906253,0.885291,0.926147,0.479942,0.0,0.0,0.7468


In [12]:
et = create_model('et')
tunedet = tune_model(et, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedet)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8395,0.9437,0.8299,0.8188,0.8243,0.6766,0.6767
1,0.8272,0.9126,0.8356,0.7922,0.8133,0.6526,0.6534
2,0.8210,0.9009,0.8082,0.7973,0.8027,0.6389,0.6389
3,0.8673,0.9318,0.8562,0.8503,0.8532,0.7321,0.7321
4,0.8421,0.9159,0.8082,0.8369,0.8223,0.6803,0.6807
Mean,0.8394,0.9210,0.8276,0.8191,0.8232,0.6761,0.6764
Std,0.0160,0.0150,0.0181,0.0223,0.0169,0.0319,0.0318


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8416,0.9310,0.8554,0.7802,0.8161,0.6775,0.6797


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.596673,0.871356,0.763590,0.339436,0.229405,0.713469,0.915850,0.766661,0.246346,0.610858,...,0.179938,0.865118,0.902924,0.897048,0.893266,0.947896,0.421826,0.0,0.0,0.9252
1,0.639353,0.816179,0.667370,0.238672,0.248595,0.651654,0.833907,0.694445,0.301305,0.651882,...,0.240837,0.815005,0.826968,0.772842,0.794202,0.828693,0.519998,0.0,1.0,0.5535
2,0.744488,0.849168,0.532340,0.483475,0.330479,0.752260,0.769390,0.770387,0.285212,0.693368,...,0.177992,0.707211,0.807656,0.783236,0.834795,0.771666,0.427091,1.0,1.0,0.7824
3,0.641973,0.903613,0.774207,0.384978,0.184514,0.737401,0.883928,0.795394,0.251806,0.653553,...,0.123020,0.854079,0.916789,0.901050,0.870574,0.934901,0.426975,0.0,0.0,0.9963
4,0.619954,0.874259,0.810200,0.624021,0.236910,0.647194,0.867252,0.754209,0.311381,0.597900,...,0.158722,0.880347,0.889378,0.807319,0.912795,0.933768,0.420251,0.0,0.0,0.5978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.799560,0.754270,0.643566,0.536195,0.172319,0.713179,0.737869,0.755434,0.322681,0.935287,...,0.155058,0.661590,0.832689,0.750921,0.818642,0.787203,0.377203,1.0,1.0,0.9224
198,0.614671,0.856247,0.783418,0.481726,0.175513,0.723720,0.897253,0.793734,0.280612,0.703699,...,0.188115,0.877789,0.921747,0.903092,0.891612,0.935127,0.468373,0.0,0.0,0.9543
199,0.633709,0.756536,0.713664,0.576160,0.206641,0.660488,0.785284,0.772561,0.316518,0.661370,...,0.169870,0.773990,0.846254,0.829869,0.834830,0.843611,0.428948,1.0,1.0,0.9002
200,0.584620,0.858842,0.725086,0.356093,0.196903,0.638933,0.923885,0.830049,0.244833,0.611456,...,0.176632,0.837904,0.910368,0.906253,0.885291,0.926147,0.479942,0.0,0.0,0.6658


In [13]:
catboost = create_model('catboost')
tunedcat1 = tune_model(catboost, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedcat1)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8611,0.9409,0.8435,0.8493,0.8464,0.7197,0.7197
1,0.8457,0.9123,0.8425,0.8200,0.8311,0.6891,0.6893
2,0.8179,0.9044,0.8014,0.7959,0.7986,0.6324,0.6325
3,0.8704,0.9296,0.8699,0.8467,0.8581,0.7388,0.7391
4,0.8359,0.9206,0.8014,0.8298,0.8153,0.6678,0.6681
Mean,0.8462,0.9215,0.8317,0.8283,0.8299,0.6896,0.6897
Std,0.0185,0.0128,0.0267,0.0195,0.0213,0.0376,0.0376


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.8663,0.9324,0.8554,0.8256,0.8402,0.7254,0.7257


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.596673,0.871356,0.763590,0.339436,0.229405,0.713469,0.915850,0.766661,0.246346,0.610858,...,0.179938,0.865118,0.902924,0.897048,0.893266,0.947896,0.421826,0.0,0.0,0.9645
1,0.639353,0.816179,0.667370,0.238672,0.248595,0.651654,0.833907,0.694445,0.301305,0.651882,...,0.240837,0.815005,0.826968,0.772842,0.794202,0.828693,0.519998,0.0,0.0,0.5290
2,0.744488,0.849168,0.532340,0.483475,0.330479,0.752260,0.769390,0.770387,0.285212,0.693368,...,0.177992,0.707211,0.807656,0.783236,0.834795,0.771666,0.427091,1.0,1.0,0.8633
3,0.641973,0.903613,0.774207,0.384978,0.184514,0.737401,0.883928,0.795394,0.251806,0.653553,...,0.123020,0.854079,0.916789,0.901050,0.870574,0.934901,0.426975,0.0,0.0,0.9898
4,0.619954,0.874259,0.810200,0.624021,0.236910,0.647194,0.867252,0.754209,0.311381,0.597900,...,0.158722,0.880347,0.889378,0.807319,0.912795,0.933768,0.420251,0.0,0.0,0.7944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.799560,0.754270,0.643566,0.536195,0.172319,0.713179,0.737869,0.755434,0.322681,0.935287,...,0.155058,0.661590,0.832689,0.750921,0.818642,0.787203,0.377203,1.0,1.0,0.9480
198,0.614671,0.856247,0.783418,0.481726,0.175513,0.723720,0.897253,0.793734,0.280612,0.703699,...,0.188115,0.877789,0.921747,0.903092,0.891612,0.935127,0.468373,0.0,0.0,0.9771
199,0.633709,0.756536,0.713664,0.576160,0.206641,0.660488,0.785284,0.772561,0.316518,0.661370,...,0.169870,0.773990,0.846254,0.829869,0.834830,0.843611,0.428948,1.0,1.0,0.9520
200,0.584620,0.858842,0.725086,0.356093,0.196903,0.638933,0.923885,0.830049,0.244833,0.611456,...,0.176632,0.837904,0.910368,0.906253,0.885291,0.926147,0.479942,0.0,0.0,0.9011


In [14]:
lightgbm = create_model('lightgbm')
tunedlightgbm = tune_model(lightgbm,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedlightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8673,0.9414,0.8503,0.8562,0.8532,0.7321,0.7321
1,0.8241,0.9120,0.8425,0.7834,0.8119,0.6471,0.6486
2,0.8241,0.9020,0.8082,0.8027,0.8055,0.6449,0.6449
3,0.8673,0.9341,0.8630,0.8456,0.8542,0.7324,0.7326
4,0.8390,0.9174,0.8014,0.8357,0.8182,0.6739,0.6743
Mean,0.8443,0.9214,0.8331,0.8247,0.8286,0.6861,0.6865
Std,0.0195,0.0144,0.0241,0.0273,0.0209,0.0391,0.0388


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8515,0.9289,0.8554,0.7978,0.8256,0.6965,0.6978


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.596673,0.871356,0.763590,0.339436,0.229405,0.713469,0.915850,0.766661,0.246346,0.610858,...,0.179938,0.865118,0.902924,0.897048,0.893266,0.947896,0.421826,0.0,0.0,0.9673
1,0.639353,0.816179,0.667370,0.238672,0.248595,0.651654,0.833907,0.694445,0.301305,0.651882,...,0.240837,0.815005,0.826968,0.772842,0.794202,0.828693,0.519998,0.0,1.0,0.6120
2,0.744488,0.849168,0.532340,0.483475,0.330479,0.752260,0.769390,0.770387,0.285212,0.693368,...,0.177992,0.707211,0.807656,0.783236,0.834795,0.771666,0.427091,1.0,1.0,0.8785
3,0.641973,0.903613,0.774207,0.384978,0.184514,0.737401,0.883928,0.795394,0.251806,0.653553,...,0.123020,0.854079,0.916789,0.901050,0.870574,0.934901,0.426975,0.0,0.0,0.9893
4,0.619954,0.874259,0.810200,0.624021,0.236910,0.647194,0.867252,0.754209,0.311381,0.597900,...,0.158722,0.880347,0.889378,0.807319,0.912795,0.933768,0.420251,0.0,0.0,0.7284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.799560,0.754270,0.643566,0.536195,0.172319,0.713179,0.737869,0.755434,0.322681,0.935287,...,0.155058,0.661590,0.832689,0.750921,0.818642,0.787203,0.377203,1.0,1.0,0.9739
198,0.614671,0.856247,0.783418,0.481726,0.175513,0.723720,0.897253,0.793734,0.280612,0.703699,...,0.188115,0.877789,0.921747,0.903092,0.891612,0.935127,0.468373,0.0,0.0,0.9855
199,0.633709,0.756536,0.713664,0.576160,0.206641,0.660488,0.785284,0.772561,0.316518,0.661370,...,0.169870,0.773990,0.846254,0.829869,0.834830,0.843611,0.428948,1.0,1.0,0.9680
200,0.584620,0.858842,0.725086,0.356093,0.196903,0.638933,0.923885,0.830049,0.244833,0.611456,...,0.176632,0.837904,0.910368,0.906253,0.885291,0.926147,0.479942,0.0,0.0,0.8989


In [22]:
from itertools import combinations
ml = [tunedrf, tunedet, tunedlightgbm, tunedcat1, tunedxgb,tunedmlp]
a2 =  list(combinations(ml, 2))
a3 =  list(combinations(ml, 3))
a4 =  list(combinations(ml, 4))
a5 =  list(combinations(ml, 5))
a6 =  list(combinations(ml, 6))
all = a2 + a3 + a4 + a5 + a6 
print(len(all))
print(len(a2),len(a3),len(a4),len(a5),len(a6))

57
15 20 15 6 1


In [16]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['Accuracy']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8642,0.9433,0.8367,0.8601,0.8483,0.7254,0.7256
1,0.8272,0.9102,0.8288,0.7961,0.8121,0.6522,0.6527
2,0.8210,0.8988,0.8082,0.7973,0.8027,0.6389,0.6389
3,0.8673,0.9302,0.8699,0.8411,0.8552,0.7328,0.7331
4,0.8266,0.9113,0.7945,0.8169,0.8056,0.6492,0.6494
Mean,0.8413,0.9188,0.8276,0.8223,0.8248,0.6797,0.6800
Std,0.0201,0.0159,0.0258,0.0250,0.0223,0.0406,0.0407


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8614,0.9308,0.8795,0.8022,0.8391,0.7178,0.7201


(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False), ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=Fa

In [23]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['AUC']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8611,0.9437,0.8571,0.8400,0.8485,0.7203,0.7204
1,0.8272,0.9131,0.8425,0.7885,0.8146,0.6531,0.6543
2,0.8210,0.9040,0.8219,0.7895,0.8054,0.6398,0.6402
3,0.8735,0.9349,0.8836,0.8431,0.8629,0.7455,0.7462
4,0.8359,0.9204,0.8288,0.8121,0.8203,0.6694,0.6695
Mean,0.8437,0.9232,0.8468,0.8146,0.8303,0.6856,0.6861
Std,0.0202,0.0144,0.0220,0.0236,0.0217,0.0405,0.0405


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8515,0.9336,0.8795,0.7849,0.8295,0.6987,0.7023


(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=11,
                       max_features=0.4867270889539521, max_leaf_nodes=None,
                       max_samples=None,
                       min_impurity_decrease=6.262714950879596e-06,
                       min_impurity_split=None, min_samples_leaf=2,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=300, n_jobs=-1, oob_score=False,
                       random_state=0, verbose=0, warm_start=False), LGBMClassifier(bagging_fraction=0.42909705448906843, bagging_freq=4,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.5821037761168325, importance_type='split',
               learning_rate=0.04375605780506392, max_depth=-1,
               min_child_samples=1, min_child_weight=0.001,
               min_split_gain=0.0010477189134533

In [25]:
blend = blend_models([tunedxgb,tunedmlp,tunedrf,tunedlightgbm,tunedcat1])
md = predict_model(blend,data = test1)
md
y_pred = md['Label']
y_true = md['active']
t = y_pred.astype(float)
t
from sklearn.metrics import balanced_accuracy_score
accuracy = balanced_accuracy_score(y_true, t)
accuracy

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8580,0.9428,0.8571,0.8344,0.8456,0.7143,0.7145
1,0.8272,0.9110,0.8425,0.7885,0.8146,0.6531,0.6543
2,0.8210,0.9046,0.8219,0.7895,0.8054,0.6398,0.6402
3,0.8673,0.9351,0.8836,0.8323,0.8571,0.7334,0.7346
4,0.8359,0.9207,0.8356,0.8079,0.8215,0.6698,0.6701
Mean,0.8419,0.9229,0.8481,0.8105,0.8289,0.6821,0.6827
Std,0.0179,0.0143,0.0210,0.0199,0.0194,0.0359,0.0360


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8374,0.9246,0.8523,0.7895,0.8197,0.6721,0.6737


0.8391798418972332

[LightGBM] [Warning] bagging_fraction is set=0.42909705448906843, subsample=1.0 will be ignored. Current value: bagging_fraction=0.42909705448906843
[LightGBM] [Warning] feature_fraction is set=0.5821037761168325, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5821037761168325
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
